<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Advanced_ML_AI/Clase_02_Optimizacion_Modelos/02B_%7C_Ejercicio_CHALLENGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 INTRODUCCIÓN
En este notebook podéis resolver el ejercicio CHALLENGE sobre churn en una empresa telco.



Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1Rzsrt7jqXUUyHNQMq20xaCfQw-YamYdZzYjfJPxZ12Y/edit?usp=sharing)
*   [Enlace a este notebook](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Advanced_ML_AI/Clase_02_Optimizacion_Modelos/02B_%7C_Ejercicio_CHALLENGE.ipynb)


## 0101 Import
Puedes importar aquí las librerías que creas que vas a necesitar.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import ensemble

## 0102 Carga el dataset del Churn Telco para ENTRENAR
Aquí teneis el url del archivo raw del dataset de churn en una empresa Telco. OJO: este es el dataset para entrenar vuestro modelo, sobre el que conocemos el target (si un cliente ha realizado o no churn). 

https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/CHURN_DF_train.csv

Importa los datos en un dataframe llamado **dataframe df**.

In [5]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/CHURN_DF_train.csv'

# Importa csv
df = pd.read_csv(url, sep=',')

# Pequeña limpieza
df.loc[df['TotalCharges']==" ", 'TotalCharges'] = np.NaN
df['TotalCharges'] = df['TotalCharges'].astype(float)

print(len(df))

# Visualización primeras filas
df.head()

5043


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,3668-QPYBK,Male,0,0,0,2,1,1,53.85,108.15,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1
1,9237-HQITU,Female,0,0,0,2,1,1,70.70,151.65,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1
2,9305-CDSKC,Female,0,0,0,8,1,1,99.65,820.50,0,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1
3,1452-KIOVK,Male,0,0,1,22,1,1,89.10,1949.40,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0
4,6713-OKOMC,Female,0,0,0,10,0,0,29.75,301.90,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0


## 0103 Carga el dataset del Churn Telco a PREDECIR

Aquí teneis el url del archivo raw del dataset de churn en una empresa Telco. OJO: este es el dataset para realizar una predicción con vuestro modelo. No conocemos el target (si un cliente ha realizado o no churn). 

https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/CHURN_X_submit.csv

Importa los datos en un dataframe llamado **dataframe df_submit**.

In [6]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/CHURN_X_submit.csv'

# Importa csv
df_submit = pd.read_csv(url, sep=',')

# Pequeña limpieza
df_submit.loc[df_submit['TotalCharges']==" ", 'TotalCharges'] = np.NaN
df_submit['TotalCharges'] = df_submit['TotalCharges'].astype(float)

print(len(df_submit))

# Visualización primeras filas
df_submit.head()

2000


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,2636-SJDOU,Female,0,0,0,34,1,1,70.00,2416.10,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0
1,8336-TAVKX,Female,1,1,0,12,1,1,79.20,943.85,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0
2,7649-PHJVR,Female,0,0,0,1,1,1,44.90,44.90,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
3,5309-TAIKL,Male,0,0,0,51,0,1,44.45,2181.55,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0
4,9052-VKDUW,Female,0,0,1,26,0,0,45.80,1147.00,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1


# 02 EDA
Para familiarizarte con los datos puedes realizar un pequeño análisis exploratorio visual. Puedes usar los métodos .shape, .info(), .describe(), por ejemplo para entender su tamaño y sus variables. 

Recuerda que puedes encontrar mis clases sobre análisis exploratorio [aquí](https://github.com/JotaBlanco/TheValley/tree/main/EDA/).

In [ ]:
df.info()

In [ ]:
df.describe()

# 03 Preparación de los datos
Normalmente el dataframe contiene nulos, así como variables categóricas y presencia de ciertas variables que probablemente no queramos usar. 

Necesitamos limpiarlo antes de poder entrenar los modelos.

### 0301 Variables Útiles
De entre las variables disponibles, quedémonos con la que queramos utilizar como predictoras para el estudio.

OJO, todas las transformaciones que hagamos sobre el dataset de entrenamiento debereis hacerlas también sobre el de submisión.

In [10]:
df.head()

In [ ]:
df.columns

In [ ]:
cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges',
       'TotalCharges', 'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No', 'StreamingTV_No',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']

cat_cols = ['gender']

In [ ]:
df = df[cols+['Churn']]
df_submit = df_submit[['customerID']+cols]

### 030102 Dumificación de variables categóricas
Recuerda que debemos transformar las variables categóricas para convertirlas en numéricas. **EN AMBOS DATASETS!**

In [ ]:
df = pd.get_dummies(df[cols+['Churn']], 
                    prefix_sep='_',
                    drop_first=True, 
                    columns=cat_cols)

display(df.shape)


df_submit = pd.get_dummies(df_submit[['customerID']+cols], 
                    prefix_sep='_',
                    drop_first=True, 
                    columns=cat_cols)

display(df_submit.shape)


df.head()

### 030103 Limpieza de nulos
Recuerda que puedes mirarlo con dataframe.isna().sum() y si sí hay nulos, puedes rellenarlos con el método .fillna().

In [ ]:
df.isna().sum()

In [ ]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].mean())
df_submit['TotalCharges'] = df_submit['TotalCharges'].fillna(df['TotalCharges'].mean())

In [ ]:
df.isna().sum()

### 030104 Train - test

In [ ]:
# Primero generamos la matriz X con las variables predictoras 
# y el vector y con la variable a predecir
X = df.drop('Churn',axis=1)
y = df['Churn']

# Comprueba sus tamaños
X.shape, y.shape

In [ ]:
# Haz la partición train-test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33,
                                                    random_state=42)

# Comprueba sus tamaños
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 04 Entrena un modelo
Entrena un modelo y evalúa su calidad. Ten en cuenta todo lo que hemos visto: calidad de la predicción, posibilidad de overfitting...
Puedes utilizar cualquier tipo de modelo de los que se han visto en el máster hasta ahora. Por ejemplo:
- Árboles de decisión: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html.
- Random Forests: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html.
- Gradient Boosting: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html.
- AdaBoost: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html.
- XGBoost: https://xgboost.readthedocs.io/en/latest/parameter.html.

# 05 Predicción
Realiza una predicción con tu mejor modelo sobre el dataset df_submit. 

Luego, quédate con dos columnas: 


*   La identificadora: 'customerID'
*   La predicción: 'prediction'


Guarda el archivo y súbelo [aquí](https://drive.google.com/drive/folders/1CWoVRO5N7lx-eTmvd4Aa94z8fS1ASrI5?usp=sharing) con el nombre de archivo identificador_personal.csv donde el identificador_personal es el nombre que aparecerá con tu predicción: puede ser tu nombre, o cualquier mote si no quieres que se sepa cuál es tu predicción.


In [10]:
ejemplo_archivo = pd.DataFrame({
    'customerID': ['3668-QPYBK', '9237-HQITU', '9305-CDSKC'], 
    'prediction': [1, 0, 0]})
ejemplo_archivo

,customerID,prediction
0,3668-QPYBK,1
1,9237-HQITU,0
2,9305-CDSKC,0
